<a href="https://colab.research.google.com/github/mirtorande/muffin-vs-chihuahua/blob/main/MuffinsChihuahuas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
! pip install -qq wandb
import wandb
from wandb.keras import WandbMetricsLogger
wandb.login()
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
from tqdm.auto import tqdm
from tensorflow.keras.utils import plot_model
import numpy as np
from sklearn.model_selection import StratifiedKFold

wandb: Currently logged in as: mirtorande (minigi). Use `wandb login --relogin` to force relogin


In [ ]:
# Define a config dictionary object
configs = dict(
    image_size = 64,
    #batch_size=1024,
    #init_learning_rate = 3e-4,
    epochs = 50,
    optimizer = 'adam',
    loss_fn = 'binary_crossentropy',
    metrics = ['acc'],
)

## Load the data: the Muffins vs Chihuahuas dataset

### Data download


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = "/content/drive/My Drive/Datasets/archive"

### Filter out the corrupted images

In [ ]:
"""num_skipped = 0
for folder_name in ('test/chihuahua', 'train/chihuahua', 'test/muffin', 'train/muffin'):
    folder_path = os.path.join(dataset_path, folder_name)
    for fname in tqdm(os.listdir(folder_path)):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)"""


'num_skipped = 0\nfor folder_name in (\'test/chihuahua\', \'train/chihuahua\', \'test/muffin\', \'train/muffin\'):\n    folder_path = os.path.join(dataset_path, folder_name)\n    for fname in tqdm(os.listdir(folder_path)):\n        fpath = os.path.join(folder_path, fname)\n        try:\n            fobj = open(fpath, "rb")\n            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)\n        finally:\n            fobj.close()\n\n        if not is_jfif:\n            num_skipped += 1\n            # Delete corrupted image\n            os.remove(fpath)\n\nprint("Deleted %d images" % num_skipped)'

### Generate a Dataset

In [ ]:
def load_datasets(config, use_val=True):
  """
  Loads Training and Test datasets
  """
  image_size = (config['image_size'], config['image_size']) # (224, 224) originally
  batch_size = config['batch_size']

  if use_val:
    train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
        dataset_path + '/train',
        image_size=image_size,
        batch_size=batch_size,
        validation_split = 0.2,
        subset='both',
        shuffle=True,
        seed=1337
    )
  else:
    train_ds = tf.keras.utils.image_dataset_from_directory(
        dataset_path + '/train',
        image_size=image_size,
        batch_size=batch_size,
        shuffle=True,
        seed=1337
    )
    val_ds = None

  test_ds = tf.keras.utils.image_dataset_from_directory(
      dataset_path + '/test',
      image_size=image_size,
      batch_size=batch_size,
      shuffle=False
  )
  return train_ds, val_ds, test_ds

## Visualize the data

In [ ]:
def print_class_counts(dataset):
  n_muf=0
  n_chi = 0
  for images, labels in tqdm(dataset):
    for label in labels:
      if label == 0:
        n_chi += 1
      else:
        n_muf += 1

  print(n_muf, n_chi)

In [ ]:
def show_sample_figures(dataset):
  plt.figure(figsize=(10, 10))
  for images, labels in dataset.take(1):
      for i in range(9):
          ax = plt.subplot(3, 3, i + 1)
          plt.imshow(images[i].numpy().astype("uint8"))
          plt.title(int(labels[i]))
          plt.axis("off")

## Using image data augmentation

In [ ]:
def augment_dataset(dataset, show_preview=False):
  data_augmentation = keras.Sequential(
      [
          layers.RandomFlip("horizontal"),
          layers.RandomRotation(0.1),
      ]
  )

  # Show augmentation preview
  if show_preview:
    plt.figure(figsize=(10, 10))
    for images, _ in dataset.take(1):
        for i in range(9):
            augmented_images = data_augmentation(images)
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(augmented_images[1].numpy().astype("uint8"))
            plt.axis("off")

  # Apply `data_augmentation` to the training images.
  dataset = dataset.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf.data.AUTOTUNE,
  )

  return dataset

In [ ]:
def prefetch_datasets(*args):
  # Prefetching samples in GPU memory helps maximize GPU utilization.
  for i in range(len(args)):
    args[i] = args[i].prefetch(tf.data.AUTOTUNE)
  return args

# Build the model

In [ ]:
def make_model(config, input_shape, num_classes):
  inputs = keras.Input(shape=input_shape)

  # Entry block
  x = layers.Rescaling(1.0 / 255)(inputs)
  x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  previous_block_activation = x  # Set aside residual

  for size in [256, 512, 728]:
      x = layers.Activation("relu")(x)
      x = layers.SeparableConv2D(size, 3, padding="same")(x)
      x = layers.BatchNormalization()(x)

      x = layers.Activation("relu")(x)
      x = layers.SeparableConv2D(size, 3, padding="same")(x)
      x = layers.BatchNormalization()(x)

      x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

      # Project residual
      residual = layers.Conv2D(size, 1, strides=2, padding="same")(
          previous_block_activation
      )
      x = layers.add([x, residual])  # Add back residual
      previous_block_activation = x  # Set aside next residual

  x = layers.SeparableConv2D(1024, 3, padding="same")(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation("relu")(x)

  x = layers.GlobalAveragePooling2D()(x)
  if num_classes == 2:
      activation = "sigmoid"
      units = 1
  else:
      activation = "softmax"
      units = num_classes

  x = layers.Dropout(config['dropout'])(x)
  outputs = layers.Dense(units, activation=activation)(x)

  return keras.Model(inputs, outputs)

# Train the model

In [ ]:
def train_model(config, train_ds, test_ds):
  model = make_model(config, input_shape=(config.image_size, config.image_size) + (3,), num_classes=2)
  keras.utils.plot_model(model, show_shapes=True)

  epochs = config.epochs

  model.compile(
      optimizer=keras.optimizers.Adam(config.init_learning_rate),
      loss=config.loss_fn,
      metrics=["accuracy"],
  )

  callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, start_from_epoch=5), WandbMetricsLogger(log_freq=2)]

  history = model.fit(
      train_ds,
      epochs=epochs,
      callbacks=callbacks,
      validation_data=test_ds,
  )
  return history

# Objective

In [ ]:
def objective(config):
  train, val, _ = load_datasets(config)
  print_class_counts(train)
  show_sample_figures(train)
  train = augment_dataset(train, show_preview=True)
  train, val = prefetch_datasets(train, val)
  history = train_model(config, train, val)
  return max(history.history['val_accuracy'])

In [ ]:
def objective_5fold(config):
  train, _, _ = load_datasets(config, use_val=False)
  print_class_counts(train)
  show_sample_figures(train)
  train = augment_dataset(train, show_preview=True)

  #TODO: trovare una versione più veloce
  train_x = np.concatenate([x for x, y in train], axis=0)
  train_y = np.concatenate([y for x, y in train], axis=0)

  accuracies = []

  for train_indexes, val_indexes in StratifiedKFold(n_splits=5).split(train_x, train_y):
    model = make_model(config, input_shape=(config.image_size, config.image_size) + (3,), num_classes=2)

    epochs = config.epochs

    model.compile(
        optimizer=keras.optimizers.Adam(config.init_learning_rate),
        loss=config.loss_fn,
        metrics=["accuracy"],
    )

    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, start_from_epoch=5), WandbMetricsLogger(log_freq=2)]

    history = model.fit(
        train_x[train_indexes], train_y[train_indexes],
        epochs=epochs,
        callbacks=callbacks,
        validation_data=(train_x[val_indexes], train_y[val_indexes]),
    )
    accuracies.append(max(history.history['val_accuracy']))

  return np.mean(accuracies)

In [ ]:
def main():
    wandb.init()
    val_accuracy = objective_5fold(wandb.config)
    wandb.log({'val_accuracy': val_accuracy})

# 2: Define the search space
sweep_configuration = {
    'method': 'bayes',
    'metric':
    {
        'goal': 'maximize',
        'name': 'val_accuracy'
    },
    'parameters':
    {
        'init_learning_rate': {
            'distribution': 'log_uniform_values', 'max': 1e-2, 'min': 1e-5
        },
        'batch_size': {
          # integers between 32 and 1024
          # with evenly-distributed logarithms
          'distribution': 'q_log_uniform_values',
          'q': 32,
          'min': 32,
          'max': 1024,
        },
        'dropout': {
          'values': [0.2, 0.3, 0.4, 0.5, 0.7]
        },
    }
}
sweep_configuration['parameters'].update({attribute:{'value':value} for attribute, value in configs.items()})

# 3: Start the sweep
sweep_id = wandb.sweep(
    sweep=sweep_configuration,
    project='MuffinChihuahuas'
)

wandb.agent(sweep_id, function=main, count=10)

Create sweep with ID: jgnvihae
Sweep URL: https://wandb.ai/minigi/MuffinChihuahuas/sweeps/jgnvihae


wandb: Agent Starting Run: fs6c3sch with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.4
wandb: 	epochs: 50
wandb: 	image_size: 64
wandb: 	init_learning_rate: 0.0005844683598220263
wandb: 	loss_fn: binary_crossentropy
wandb: 	metrics: ['acc']
wandb: 	optimizer: adam


Run fs6c3sch errored: NameError("name 'objective_5fold' is not defined")
wandb: ERROR Run fs6c3sch errored: NameError("name 'objective_5fold' is not defined")
wandb: Agent Starting Run: nnh3d7y1 with config:
wandb: 	batch_size: 576
wandb: 	dropout: 0.4
wandb: 	epochs: 50
wandb: 	image_size: 64
wandb: 	init_learning_rate: 0.0007756209000089389
wandb: 	loss_fn: binary_crossentropy
wandb: 	metrics: ['acc']
wandb: 	optimizer: adam


Run nnh3d7y1 errored: NameError("name 'objective_5fold' is not defined")
wandb: ERROR Run nnh3d7y1 errored: NameError("name 'objective_5fold' is not defined")
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: rtqsqprm with config:
wandb: 	batch_size: 992
wandb: 	dropout: 0.7
wandb: 	epochs: 50
wandb: 	image_size: 64
wandb: 	init_learning_rate: 5.589937577247177e-05
wandb: 	loss_fn: binary_crossentropy
wandb: 	metrics: ['acc']
wandb: 	optimizer: adam


Run rtqsqprm errored: NameError("name 'objective_5fold' is not defined")
wandb: ERROR Run rtqsqprm errored: NameError("name 'objective_5fold' is not defined")
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


Getting best values

In [ ]:
api = wandb.Api()
sweep = api.sweep(f"minigi/MuffinChihuahuas/sweeps/hmwv6nzd")

# Get best run parameters
best_run = sweep.best_run(order='val_accuracy')
best_parameters = best_run.config
print(best_parameters)

wandb: Sorting runs by -summary_metrics.val_accuracy


{'epochs': 50, 'dropout': 0.1, 'loss_fn': 'binary_crossentropy', 'metrics': ['acc'], 'optimizer': 'adam', 'batch_size': 32, 'image_size': 64, 'init_learning_rate': 0.0010611468952407492}


In [ ]:
best_run.summary

{'epoch/epoch': 9, 'batch/batch_step': 1198, '_runtime': 81.629061460495, 'epoch/loss': 0.16502374410629272, 'epoch/val_accuracy': 0.9048625826835632, '_step': 610, 'batch/loss': 0.16502374410629272, 'epoch/accuracy': 0.9355690479278564, '_timestamp': 1689979253.0471263, 'batch/accuracy': 0.9355690479278564, 'epoch/val_loss': 0.2531700134277344, 'batch/learning_rate': 0.0010611468460410831, 'epoch/learning_rate': 0.0010611468460410831, '_wandb': {'runtime': 78}, 'val_accuracy': 0.920718789100647}

In [ ]:
! wandb sweep --stop minigi/MuffinChihuahuas/hmwv6nzd

wandb: Stopping sweep minigi/MuffinChihuahuas/hmwv6nzd
Traceback (most recent call last):
  File "/usr/local/bin/wandb", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/decorators.py", line 33, in new_func
    return f(get_current_context(), *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/cli

In [ ]:
wandb.init(project='MuffinChihuahua', config=best_parameters)
print(wandb.config)
finale_value = objective_5fold(wandb.config)
print(final_value)
wandb.finish()

wandb: WARNING Ignored wandb.init() arg project when running a sweep.


{'batch_size': 992, 'dropout': 0.7, 'epochs': 50, 'image_size': 64, 'init_learning_rate': 5.589937577247177e-05, 'loss_fn': 'binary_crossentropy', 'metrics': ['acc'], 'optimizer': 'adam'}
Found 4733 files belonging to 2 classes.
Found 1184 files belonging to 2 classes.


  0%|          | 0/5 [00:00<?, ?it/s]

2174 2559


# Run inference on new data

In [ ]:
"""def keralize_img(path):
  img = keras.utils.load_img(
    path, target_size=configs.image_size
  )
  img_array = keras.utils.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)  # Create batch axis
  return img_array

chihuahua_img = keralize_img(dataset_path + '/test/muffin/img_0_67.jpg')
muffin_img = keralize_img(dataset_path + '/test/chihuahua/img_0_1107.jpg')

def predict(img):
  model.load_weights("/content/save_at_7.keras")
  predictions = model.predict(img)
  score = float(predictions[0])
  print(f"This image is {100 * (1 - score):.2f}% muffin and {100 * score:.2f}% chihuahua.")

predict(chihuahua_img)
predict(muffin_img)"""